In [1]:
# Taken from main06.cc Pythia example. A study of event properties of LEP1 events

from numpythia import Pythia, hepmc_write, hepmc_read
from numpythia import STATUS, HAS_END_VERTEX, ABS_PDG_ID

from pyjet import cluster, DTYPE_PTEPM, DTYPE_EP
from pyjet.testdata import get_event

import numpy as np
import sys

from pyjet import cluster
from pyjet import ClusterSequenceArea, JetDefinition, ClusterSequence

from dwave.system.samplers import DWaveSampler
from dwave.system.composites import EmbeddingComposite

In [2]:
# Pythia generator settings. Process selection e+e- collision.
# WeakSingleBoson:ffbar2gmZ : Produce Z bosons and interference with photon
# Consider only Z (PDGID=23) decays to light quarks

params = {"Beams:idA" : "11", "Beams:idB" : "-11","Beams:eCM": 91.1876, "WeakSingleBoson:ffbar2gmZ" : "on", "23:onMode": "off", "23:onIfAny": "1 2 3 4 5", "PDF:lepton": "off"}
pythia = Pythia(params =params, random_state=1)

# Consider only final-state particles
selection = ((STATUS == 1) & ~HAS_END_VERTEX)

In [3]:
# Array of pseudo-jets or particles to cluster
particles = []
array = []

# Begin event loop. Generate events
for event in pythia(events=1):
    array = event.all(selection)

In [4]:
#For testing: filter out particles by energy
particles = array#[array["E"] > 2.0]

In [5]:
print(len(particles))

35


In [6]:
len(array)

35

In [7]:
# Ep - format for reference, not for plotting
with open('ee_particles.dat', 'w') as f:
    for p in particles:
        f.write("%s %s %s %s\n" %( p['E'], p['px'], p['py'], p['pz']))


In [8]:
# Print out each momentum (starting from 0 to enable gnuplot to easily draw a line)

with open('out.particles', 'w') as f:
    for p in particles:
        f.write("0 0 0 0\n")
        f.write("%s %s %s %s\n\n\n" %( p['px'], p['py'], p['pz'], p['E']))


In [9]:
# Cluster particles into jets
# First generate a whole "clustering sequence" with the e+e- kt algorithm

jetdef = JetDefinition('ee_kt')
sequence = ClusterSequence(particles, jetdef, ep=True)

In [10]:
# Then select the jets that come out of clustering 
jets = sequence.exclusive_jets(3)

In [11]:
with open('out.jetconst', 'w') as f:
    for c in range(len(jets[0].constituents_array(ep=True))):
        f.write("0 0 0 0\n")
        f.write("%s %s %s %s\n\n\n" %(jets[0].constituents_array(ep=True)[c]['px'], jets[0].constituents_array(ep=True)[c]['py'], jets[0].constituents_array(ep=True)[c]['pz'], jets[0].constituents_array(ep=True)[c]['E']))

In [12]:
# Print out the jets to a file
with open('out.ktjets', 'w') as f:
    for jet in jets:
        f.write("0 0 0 0\n")
        f.write("%s %s %s %s\n\n\n" %(jet.px, jet.py, jet.pz, jet.e))

In [13]:
# Create qubo
from pyqubo import Spin, Array

# Create an array os spin variables
n_part = len(particles) 
s = Array.create('s', shape=n_part, vartype='BINARY')

# Create an array of zeroes for qubo matrix coefficients
coeff = [[0] * n_part for _ in range(n_part)] 
theta = np.pi/4.

for i in range(0,n_part):
    for j in range(0,n_part):
        coeff[i][j] = (particles[i]['px']*particles[j]['px'] + particles[i]['py']*particles[j]['py'] + particles[i]['pz']*particles[j]['pz'] - particles[i]['E']*particles[j]['E'] * np.cos(theta))/(1 - np.cos(theta))

# Construct Hamiltonian
H = sum([ -1.0*coeff[i][j]*s[i]*s[j] for i in range(0,n_part) for j in range(0,n_part)])

# Compile model using pyqubo

model = H.compile()
qubo, offset = model.to_qubo()

In [26]:
# Study sparsity of QUBO
import dimod
import hybrid

bqm = dimod.BinaryQuadraticModel.from_qubo(qubo, offset=0.0)

In [27]:
print("BQM: {} nodes, {} edges, {:.2f} density". format( len(bqm), len(bqm.quadratic), hybrid.bqm_density(bqm)))

BQM: 35 nodes, 595 edges, 1.00 density


In [16]:
# Submit qubo to D-Wave

sampler = EmbeddingComposite(DWaveSampler(solver='Advantage_system1.1'))
response = sampler.sample_qubo(qubo, num_reads=1)

In [17]:
qpu_energies = []
for datum in response.data(['sample', 'energy']):
    qpu_energies.append(datum.energy)
    if (datum.energy < 0):
        print(datum.sample, datum.energy)

In [19]:
qpu_energies

[820.4262289060534]

In [20]:
for datum in response.data(['sample', 'energy']):
    print(datum.sample, datum.energy)

{'s[0]': 1, 's[10]': 0, 's[11]': 1, 's[12]': 1, 's[13]': 0, 's[14]': 1, 's[15]': 0, 's[16]': 0, 's[17]': 1, 's[18]': 0, 's[19]': 1, 's[1]': 1, 's[20]': 0, 's[21]': 1, 's[22]': 1, 's[23]': 0, 's[24]': 0, 's[25]': 1, 's[26]': 0, 's[27]': 1, 's[28]': 1, 's[29]': 0, 's[2]': 0, 's[30]': 1, 's[31]': 0, 's[32]': 0, 's[33]': 1, 's[34]': 1, 's[3]': 0, 's[4]': 0, 's[5]': 1, 's[6]': 0, 's[7]': 0, 's[8]': 0, 's[9]': 1} 820.4262289060534


In [21]:
solution = {'s[0]': 1, 's[10]': 0, 's[11]': 1, 's[12]': 1, 's[13]': 0, 's[14]': 1, 's[15]': 0, 's[16]': 0, 's[17]': 1, 's[18]': 0, 's[19]': 1, 's[1]': 1, 's[20]': 0, 's[21]': 1, 's[22]': 1, 's[23]': 0, 's[24]': 0, 's[25]': 1, 's[26]': 0, 's[27]': 1, 's[28]': 1, 's[29]': 0, 's[2]': 0, 's[30]': 1, 's[31]': 0, 's[32]': 0, 's[33]': 1, 's[34]': 1, 's[3]': 0, 's[4]': 0, 's[5]': 1, 's[6]': 0, 's[7]': 0, 's[8]': 0, 's[9]': 1}

In [22]:
with open('out.particlesqujet', 'w') as f:
    for i in range(len(particles)):
        key = 's[%s]' %i
        if(solution[key]==1):
            f.write("0 0 0 0\n")
            f.write("%s %s %s %s\n\n\n" %( particles[i]['px'], particles[i]['py'], particles[i]['pz'], particles[i]['E']))

In [23]:
# Print out each momentum (starting from 0 to enable gnuplot to easily draw a line)
with open('ee_particles_qpu.dat', 'w') as f:
    for i in range(len(particles)):
        key = 's[%s]' %i
        f.write("%s %s %s %s %s\n" %( particles[i]['E'],particles[i]['px'], particles[i]['py'], particles[i]['pz'], solution[key]))


In [ ]:
import pyqubo
decoded_solution, broken, energy = model.decode_solution(solution, vartype='BINARY')

In [ ]:
import dimod 
solution_dimod = dimod.ExactSolver().sample_qubo(qubo)

In [ ]:
solution_dimod.lowest()

In [ ]:
dimod_energies = []
for datum in solution_dimod.data(['sample', 'energy']):
    dimod_energies.append(datum.energy)
    if (datum.energy < -1000):
        print(datum.sample, datum.energy)

In [ ]:
dimod_energies[8191]

In [ ]:
dimod_list = {'s[0]': 0, 's[10]': 1, 's[11]': 1, 's[12]': 1, 's[1]': 0, 's[2]': 0, 's[3]': 0, 's[4]': 0, 's[5]': 1, 's[6]': 1, 's[7]': 1, 's[8]': 1, 's[9]': 1}

In [ ]:
with open('out.particlesdimod', 'w') as f:
    for i in range(0,len(particles)):
        key = 's[%s]' %i
        if(dimod_list[key]==1):
            f.write("0 0 0 0\n")
            f.write("%s %s %s %s\n\n\n" %( particles[i]['px'], particles[i]['py'], particles[i]['pz'], particles[i]['E']))

In [ ]:
# Print out each momentum (starting from 0 to enable gnuplot to easily draw a line)
with open('ee_particles_dimod.dat', 'w') as f:
    for i in range(len(particles)):
        key = 's[%s]' %i
        f.write("%s %s %s %s %s\n" %( particles[i]['E'],particles[i]['px'], particles[i]['py'], particles[i]['pz'], dimod_list[key]))
